In [ ]:
from time import time
import configparser
import matplotlib.pyplot as plt
import pandas as pd
import json
import boto3

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("CLUSTER","DWH_DB")
DWH_DB_USER            = config.get("CLUSTER","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER","DWH_DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

In [ ]:
ec2 = boto3.resource('ec2', 
                   region_name='us-east-1',
                    aws_access_key_id=KEY, 
                    aws_secret_access_key=SECRET)

s3 = boto3.resource('s3', 
                    region_name='us-east-1', 
                    aws_access_key_id=KEY, 
                    aws_secret_access_key=SECRET)

iam = boto3.resource('iam', 
                   region_name='us-east-1', 
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET)

redshift = boto3.client('redshift', 
                      region_name='us-east-1',
                    aws_access_key_id=KEY, 
                    aws_secret_access_key=SECRET)

In [ ]:
for vpc in ec2.vpcs.all():
    print(vpc)

In [ ]:
ec2_client = boto3.client('ec2',
            region_name='us-east-1',
            aws_access_key_id=KEY, 
            aws_secret_access_key=SECRET)

vpc_id = ec2_client.describe_security_groups()['SecurityGroups'][0]['GroupId']

## STEP 1: IAM ROLE
- Creates an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [ ]:
# TODO: Create the IAM role
policy_document = {'Statement': [{'Action': 'sts:AssumeRole',
                                 'Effect': 'Allow',
                                 'Principal': {'Service': 'redshift.amazonaws.com'}}],
                  'Version': '2012-10-17'}
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
    Path='/',
    RoleName=DWH_IAM_ROLE_NAME,
    Description='Allows Redshigt cluster to call AWS services.',
    AssumeRolePolicyDocument=json.dumps(policy_document)
    )
    

except Exception as e:
    print(e)

In [ ]:
# TODO: Attach Policy
print('1.2 Attaching Policy')
iam_client=boto3.client('iam',
                    region_name='us-east-1', 
                    aws_access_key_id=KEY, 
                    aws_secret_access_key=SECRET
)

response = iam_client.get_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
)



iam_client.attach_role_policy(
    RoleName=DWH_IAM_ROLE_NAME,
    PolicyArn=response['Policy']['Arn']
)

In [ ]:
roleArn = iam_client.get_role(
    RoleName=DWH_IAM_ROLE_NAME
)

print(roleArn['Role']['Arn'])

## STEP 2:  Redshift Cluster

- Creates a RedShift Cluster

In [ ]:
print('DWH_CLUSTER_TYPE: ' + DWH_CLUSTER_TYPE)      
print('DWH_NUM_NODES: ' + DWH_NUM_NODES)
print('DWH_NODE_TYPE: ' + DWH_NODE_TYPE)
print('DWH_CLUSTER_IDENTIFIER: ' + DWH_CLUSTER_IDENTIFIER)
print('DWH_DB: ' + DWH_DB)  
print('DWH_DB_USER: ' + DWH_DB_USER)
print('DWH_DB_PASSWORD: ' + DWH_DB_PASSWORD)
print('DWH_PORT: ' + DWH_PORT) 

In [ ]:
try:
    response = redshift.create_cluster(
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        NumberOfNodes=int(DWH_NUM_NODES),
        Port=int(DWH_PORT),
        IamRoles=[roleArn['Role']['Arn']],
        VpcSecurityGroupIds=[vpc_id]
)
         
except Exception as e:
    print(e)

## 2.1 *Describe* the cluster to see its status
- run this block several times until the cluster status becomes `Available`

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId', 'ClusterAvailabilityStatus']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

<h2> 2.2 Take note of the cluster <font color='red'> endpoint and role ARN </font> </h2>

<font color='red'>DO NOT RUN THIS unless the cluster status becomes "Available" </font>

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

## STEP 3: Open an incoming  TCP port to access the cluster ednpoint

In [ ]:
# securitygroup = ec2.create_security_group(GroupName='test', Description='only allow SSH traffic', VpcId=vpc.id)
# securitygroup.authorize_ingress(CidrIp='0.0.0.0/0', IpProtocol='tcp', FromPort=22, ToPort=22)

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,  # TODO: fill out
        CidrIp='0.0.0.0/0',  # TODO: fill out
        IpProtocol='TCP',  # TODO: fill out
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

## STEP 4: Make sure you can connect to the clusterConnect to the cluster

In [ ]:
%load_ext sql

In [ ]:
print(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

## STEP 5: Clean up your resources

In [ ]:
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

- run this block several times until the cluster really deleted

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
iam_client.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam_client.delete_role(RoleName=DWH_IAM_ROLE_NAME)